Langchain의 Rag 컴포넌트 - 문서 로더(Document Loader)

학습 목표
1. LangChain의 Document Loader개념과 BaseLoader 인터페이스를 이해한다.
2. PDF, Web, JSON, CSV 등 다양한 형식의 문서를 로드할 수 있다.
3. 각 로더의 매개변수를 조정하여 원하는 형태로 데이터를 추출할 수 있다.
4. 메타데이터를 커스터마이징하여 문서 관리 효율성을 높일 수 있다.

사전준비
필수 라이브러리 설치
uv add langchain-community beautifulsoup4 pypdf jq

다양한 문서 형식 처리하기
- 역할 : Document Loader는 다양한 소스에서 문서를 로드
- 구현
	- Document Loader는 BaseLoader 인터페이스를 통해서 구현
	- `.load()` 또는 `.lazy_load()` 메서드를 통해 동일한 방식으로 호출
	- 대용량 데이터셋의 경우 메모리 효율을 위해 `.lazy_load()`를 사용하는 것을 권장
- 종류
	- pdf 파일 로더 > 예시 O
	- 웹 페이지 로더 > 예시 O
	- JSON 데이터 로더 > 예시 O
	- CSV 데이터 로더
	- 디렉토리 로더
	- HTML 데이터 로더
	- Markdown 데이터 로더
	- Microsoft Office 데이터 로더


In [1]:
# ENV 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# 기본 라이브러리
import os
from glob import glob

from pprint import pprint
import json

In [21]:
# PDF 파일 로더
from langchain_community.document_loaders import PyPDFLoader

# PDF 로더 초기화
# mode = 'single' > 15개의 페이지를 하나로 읽어온다.
pdf_loader = PyPDFLoader('../data/transformer.pdf', mode='single')

# 동기 로딩
pdf_docs = pdf_loader.load()
print(f"pdf 문서 개수 : ", len(pdf_docs))

# 첫번째 문서의 내용 출력
pprint(pdf_docs[0].page_content)

# 첫 번째 문서의 메타데이터 출력
print(pdf_docs[0].metadata)


pdf 문서 개수 :  1
('Provided proper attribution is provided, Google hereby grants permission to\n'
 'reproduce the tables and figures in this paper solely for use in '
 'journalistic or\n'
 'scholarly works.\n'
 'Attention Is All You Need\n'
 'Ashish Vaswani∗\n'
 'Google Brain\n'
 'avaswani@google.com\n'
 'Noam Shazeer∗\n'
 'Google Brain\n'
 'noam@google.com\n'
 'Niki Parmar∗\n'
 'Google Research\n'
 'nikip@google.com\n'
 'Jakob Uszkoreit∗\n'
 'Google Research\n'
 'usz@google.com\n'
 'Llion Jones∗\n'
 'Google Research\n'
 'llion@google.com\n'
 'Aidan N. Gomez∗ †\n'
 'University of Toronto\n'
 'aidan@cs.toronto.edu\n'
 'Łukasz Kaiser ∗\n'
 'Google Brain\n'
 'lukaszkaiser@google.com\n'
 'Illia Polosukhin∗ ‡\n'
 'illia.polosukhin@gmail.com\n'
 'Abstract\n'
 'The dominant sequence transduction models are based on complex recurrent or\n'
 'convolutional neural networks that include an encoder and a decoder. The '
 'best\n'
 'performing models also connect the encoder and decoder through an att

In [9]:
print('--------------비동기 로딩 ------------------')

# 비동기 로딩 (대용량 파일에 권장)
# 주의 : Jupyter Notebook에서는 IPython7.0+ 필요
# 일반 Python 스크립트에서는 async 함수 내에서 사용해야 함
async for page in pdf_loader.alazy_load():
	# 페이지별 처리
	print('page.metadata : ',page.metadata) # 메타데이터 출력
	print('page.page_content : ', page.page_content) # 페이지 내용 출력
	print('-'*80)

--------------비동기 로딩 ------------------
page.metadata :  {'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/transformer.pdf', 'total_pages': 15}
page.page_content :  Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser ∗
Google Brain
lukasz

In [15]:
# 웹 문서 로더
import os
# USER_AGENT 설정 (선택사항이지만 권장)
os.environ['USER_AGENT'] = 'Mozilla/5.0 (Educational Purpose)'

from langchain_community.document_loaders import WebBaseLoader

# 기본적인 텍스트 추출
web_loader = WebBaseLoader(
	web_path=[
		"https://python.langchain.com/", 
    "https://js.langchain.com/",
	]
)

# 동기 로딩
web_docs = web_loader.load()

print('len(web_docs) : ',len(web_docs))

print('web_docs[0].metadata : ', web_docs[0].metadata)

print('web_docs[0].page_content : ', web_docs[0].page_content)


len(web_docs) :  2
web_docs[0].metadata :  {'source': 'https://python.langchain.com/', 'title': 'LangChain overview - Docs by LangChain', 'description': 'LangChain is an open source framework with a pre-built agent architecture and integrations for any model or tool — so you can build agents that adapt as fast as the ecosystem evolves', 'language': 'en'}
web_docs[0].page_content :  LangChain overview - Docs by LangChainSkip to main contentDocs by LangChain home pageOpen sourceSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangChain overviewDeep AgentsLangChainLangGraphIntegrationsLearnReferenceContributePythonOverviewGet startedInstallQuickstartChangelogPhilosophyCore componentsAgentsModelsMessagesToolsShort-term memoryStreamingStructured outputMiddlewareOverviewPrebuilt middlewareCustom middlewareAdvanced usageGuardrailsRuntimeContext engineeringModel Context Protocol (MCP)Human-in-the-loopMulti-agentRetrievalLong-term memoryAgent developmentLangSmith StudioTestAg

In [ ]:
# 특정 HTML 요소만 파싱하고 싶을 경우 (bs4 활용)
import bs4

web_loader = WebBaseLoader(
	web_path=[
		"https://python.langchain.com/", 
	],
	bs_kwargs={
		#"parse_only": bs4.SoupStrainer(class_='theme-doc-markdown markdown')
	},
	bs_get_text_kwargs= {
		"separator": "|", # 구분자
		"strip": True # 공백 제거 - 불필요한 줄바꿈이나 공백을 제거해 데이터 품질을 높인다.
	}
)

web_docs = web_loader.load()

print(web_docs)

[Document(metadata={'source': 'https://python.langchain.com/', 'title': 'LangChain overview - Docs by LangChain', 'description': 'LangChain is an open source framework with a pre-built agent architecture and integrations for any model or tool — so you can build agents that adapt as fast as the ecosystem evolves', 'language': 'en'}, page_content='LangChain overview - Docs by LangChain|Skip to main content|Docs by LangChain|home page|Open source|Search...|⌘|K|Ask AI|GitHub|Try LangSmith|Try LangSmith|Search...|Navigation|LangChain overview|Deep Agents|LangChain|LangGraph|Integrations|Learn|Reference|Contribute|Python|Overview|Get started|Install|Quickstart|Changelog|Philosophy|Core components|Agents|Models|Messages|Tools|Short-term memory|Streaming|Structured output|Middleware|Overview|Prebuilt middleware|Custom middleware|Advanced usage|Guardrails|Runtime|Context engineering|Model Context Protocol (MCP)|Human-in-the-loop|Multi-agent|Retrieval|Long-term memory|Agent development|LangSmith

3. JSON 파일 로더
- 설치 : pip install jq 또는 uv add jq
- jq 스키마 주요 패턴 :
	- `.messagesp[]`: message 배열의 모든 요소
	- `.message[].content`: 각 요소의 content 필드만
	- `.`: 전체 객체 선택
- 참고 : jq 문법 확인 가능한 [링크](https://jqlang.github.io/jq/manual/)
- text_content 매개변수 설명
	- `text_content=True`: jq_schema로 추출한 결과를 문자열로 직접 사용
		(단순 텍스트 필드 추출 시)
	- `text_content=False`: jq_schema로 추출한 결과를 JSON객체러 처리
		(복잡한 구조 추출 시)

In [50]:
from langchain_community.document_loaders import JSONLoader

json_loader = JSONLoader(
	file_path="../data/kakao_chat.json",
	jq_schema=".messages[]",
	text_content=False
)

json_docs = json_loader.load()

print('처음 문서의 내용 : ', json_docs[0].page_content)
print('실제 입력 데이터 확인 : ', json_docs[4].page_content)
json_docs

처음 문서의 내용 :  {"sender": "\uae40\ucca0\uc218", "timestamp": "2023-09-15 09:30:22", "content": "\uc548\ub155\ud558\uc138\uc694 \uc5ec\ub7ec\ubd84, \uc624\ub298 \ud68c\uc758 \uc2dc\uac04 \ud655\uc778\ucc28 \uc5f0\ub77d\ub4dc\ub9bd\ub2c8\ub2e4."}
실제 입력 데이터 확인 :  {"sender": "\uae40\ucca0\uc218", "timestamp": "2023-09-15 09:34:55", "content": "\ub124, \ubaa8\ub450 \uc900\ube44\ud588\uc2b5\ub2c8\ub2e4. \ud68c\uc758 \ub54c \ubd59\uaca0\uc2b5\ub2c8\ub2e4 :)"}


[Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.json', 'seq_num': 1}, page_content='{"sender": "\\uae40\\ucca0\\uc218", "timestamp": "2023-09-15 09:30:22", "content": "\\uc548\\ub155\\ud558\\uc138\\uc694 \\uc5ec\\ub7ec\\ubd84, \\uc624\\ub298 \\ud68c\\uc758 \\uc2dc\\uac04 \\ud655\\uc778\\ucc28 \\uc5f0\\ub77d\\ub4dc\\ub9bd\\ub2c8\\ub2e4."}'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.json', 'seq_num': 2}, page_content='{"sender": "\\uc774\\uc601\\ud76c", "timestamp": "2023-09-15 09:31:05", "content": "\\ub124, \\uc548\\ub155\\ud558\\uc138\\uc694. \\uc624\\ud6c4 2\\uc2dc\\uc5d0 \\ud558\\uae30\\ub85c \\ud588\\uc5b4\\uc694."}'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.json', 'seq_num': 3}, page_content='{"sender": "\\ubc15\\ubbfc\\uc218", "timestamp": "2023-09-15 09:32:18", "content": "\\ud6

In [32]:
jq_schema = '.messages[] | .sender + ": " + .content'

loader = JSONLoader(
    file_path="../data/kakao_chat.json",
    jq_schema=jq_schema,
    text_content=True,
)

loader_docs=loader.load()

loader_docs

[Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.json', 'seq_num': 1}, page_content='김철수: 안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다.'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.json', 'seq_num': 2}, page_content='이영희: 네, 안녕하세요. 오후 2시에 하기로 했어요.'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.json', 'seq_num': 3}, page_content='박민수: 확인했습니다. 회의실은 어디인가요?'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.json', 'seq_num': 4}, page_content='이영희: 3층 대회의실입니다. 프로젝트 진행 상황 정리해오시는 거 잊지 마세요!'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.json', 'seq_num': 5}, page_content='김철수: 네, 모두 준비했습니다. 회의 때 뵙겠습니다 :)')]

In [ ]:
# 유니코드 디코딩 (한글 문자들이 유니코드 이스케이프 시퀀스로 인코딩 되어 있음)
# page content의 "sender": "\\uae40\\ucca0\\uc218" 이렇게 되어있는 부분을 한글로 디코딩 하는 방법
from langchain_core.documents import Document

json_loader = JSONLoader(
    file_path="../data/kakao_chat.json",
    jq_schema=".messages[]",     # messages 배열의 모든 아이템을 추출 - 하나의 객체를 문서로 전환
    text_content=False,          # 추출하려는 필드가 텍스트인지 여부 - 텍스트가 아닌 것은 강제로 string으로 변경
)

json_docs = json_loader.load()

decoded_json_docs = []

for doc in json_docs:
	decoded_data = json.loads(doc.page_content)
	document_obj = Document(page_content=json.dumps(decoded_data, ensure_ascii=False), metadata=doc.metadata)
	decoded_json_docs.append(document_obj)

decoded_json_docs


[Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.json', 'seq_num': 1}, page_content='{"sender": "김철수", "timestamp": "2023-09-15 09:30:22", "content": "안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다."}'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.json', 'seq_num': 2}, page_content='{"sender": "이영희", "timestamp": "2023-09-15 09:31:05", "content": "네, 안녕하세요. 오후 2시에 하기로 했어요."}'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.json', 'seq_num': 3}, page_content='{"sender": "박민수", "timestamp": "2023-09-15 09:32:18", "content": "확인했습니다. 회의실은 어디인가요?"}'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.json', 'seq_num': 4}, page_content='{"sender": "이영희", "timestamp": "2023-09-15 09:33:40", "content": "3층 대회의실입니다. 프로젝트 진행 상황 정리해오시는 거 잊지 마세요!"}'),
 Document(

In [3]:
# JSONL 파일 로드하기
from langchain_community.document_loaders import JSONLoader

json_loader = JSONLoader(
	file_path = '../data/kakao_chat.jsonl',
	jq_schema='.content',
	json_lines=True
)

json_docs = json_loader.load()

json_docs

/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.jsonl', 'seq_num': 1}, page_content='안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다.'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.jsonl', 'seq_num': 2}, page_content='네, 안녕하세요. 오후 2시에 하기로 했어요.'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.jsonl', 'seq_num': 3}, page_content='확인했습니다. 회의실은 어디인가요?'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.jsonl', 'seq_num': 4}, page_content='3층 대회의실입니다. 프로젝트 진행 상황 정리해오시는 거 잊지 마세요!'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.jsonl', 'seq_num': 5}, page_content='네, 모두 준비했습니다. 회의 때 뵙겠습니다 :)')]

In [6]:
# JSON 파일 로드하기
json_loader = JSONLoader(
	file_path="../data/kakao_chat.jsonl",
	jq_schema=".", # 전체
	json_lines=True,
	text_content=False, # false인 경우 전체로드, 위의 content_key를 주석 해제 시 true 처리 필요
)

json_docs = json_loader.load()

json_docs

[Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.jsonl', 'seq_num': 1}, page_content='{"sender": "\\uae40\\ucca0\\uc218", "timestamp": "2023-09-15 09:30:22", "content": "\\uc548\\ub155\\ud558\\uc138\\uc694 \\uc5ec\\ub7ec\\ubd84, \\uc624\\ub298 \\ud68c\\uc758 \\uc2dc\\uac04 \\ud655\\uc778\\ucc28 \\uc5f0\\ub77d\\ub4dc\\ub9bd\\ub2c8\\ub2e4."}'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.jsonl', 'seq_num': 2}, page_content='{"sender": "\\uc774\\uc601\\ud76c", "timestamp": "2023-09-15 09:31:05", "content": "\\ub124, \\uc548\\ub155\\ud558\\uc138\\uc694. \\uc624\\ud6c4 2\\uc2dc\\uc5d0 \\ud558\\uae30\\ub85c \\ud588\\uc5b4\\uc694."}'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.jsonl', 'seq_num': 3}, page_content='{"sender": "\\ubc15\\ubbfc\\uc218", "timestamp": "2023-09-15 09:32:18", "content": "\\

In [ ]:
# JSON 파일 로드하기
json_loader = JSONLoader(
	file_path="../data/kakao_chat.jsonl",
	jq_schema=".", # 전체
	content_key="content", # 일부만 로드 해야할 때 content_key 설정
	json_lines=True,
	# text_content=False, # false인 경우 전체로드, 위의 content_key를 주석 해제 시 true 처리 필요
	text_content=True, # false인 경우 전체로드, 위의 content_key를 주석 해제 시 true 처리 필요
)

json_docs = json_loader.load()

json_docs


[Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.jsonl', 'seq_num': 1}, page_content='안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다.'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.jsonl', 'seq_num': 2}, page_content='네, 안녕하세요. 오후 2시에 하기로 했어요.'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.jsonl', 'seq_num': 3}, page_content='확인했습니다. 회의실은 어디인가요?'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.jsonl', 'seq_num': 4}, page_content='3층 대회의실입니다. 프로젝트 진행 상황 정리해오시는 거 잊지 마세요!'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.jsonl', 'seq_num': 5}, page_content='네, 모두 준비했습니다. 회의 때 뵙겠습니다 :)')]

In [2]:
# 다른 속성을 metadata로 추출하는 경우
# Define the metadata extraction function
from langchain_community.document_loaders import JSONLoader

def metadata_func(record: dict, metadata: dict) -> dict:
	metadata["sender"] = record.get("sender")
	metadata["timestamp"] = record.get("timestamp")
	return metadata

loader = JSONLoader(
	file_path="../data/kakao_chat.json",
	jq_schema=".messages[]",
	content_key="content",
	metadata_func=metadata_func
)

docs = loader.load()

docs

/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.json', 'seq_num': 1, 'sender': '김철수', 'timestamp': '2023-09-15 09:30:22'}, page_content='안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다.'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.json', 'seq_num': 2, 'sender': '이영희', 'timestamp': '2023-09-15 09:31:05'}, page_content='네, 안녕하세요. 오후 2시에 하기로 했어요.'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.json', 'seq_num': 3, 'sender': '박민수', 'timestamp': '2023-09-15 09:32:18'}, page_content='확인했습니다. 회의실은 어디인가요?'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.json', 'seq_num': 4, 'sender': '이영희', 'timestamp': '2023-09-15 09:33:40'}, page_content='3층 대회의실입니다. 프로젝트 진행 상황 정리해오시는 거 잊지 마세요!'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_

In [3]:
from langchain_community.document_loaders import JSONLoader

def metadata_func(record: dict, metadata: dict) -> dict:
	metadata["sender"] = record.get("sender")
	metadata["timestamp"] = record.get("timestamp")
	return metadata


json_loader=JSONLoader(
	file_path="../data/kakao_chat.jsonl",
	jq_schema=".",
	content_key="content",
	json_lines=True,
	metadata_func=metadata_func
)

docs = json_loader.load()
docs

[Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.jsonl', 'seq_num': 1, 'sender': '김철수', 'timestamp': '2023-09-15 09:30:22'}, page_content='안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다.'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.jsonl', 'seq_num': 2, 'sender': '이영희', 'timestamp': '2023-09-15 09:31:05'}, page_content='네, 안녕하세요. 오후 2시에 하기로 했어요.'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.jsonl', 'seq_num': 3, 'sender': '박민수', 'timestamp': '2023-09-15 09:32:18'}, page_content='확인했습니다. 회의실은 어디인가요?'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/data/kakao_chat.jsonl', 'seq_num': 4, 'sender': '이영희', 'timestamp': '2023-09-15 09:33:40'}, page_content='3층 대회의실입니다. 프로젝트 진행 상황 정리해오시는 거 잊지 마세요!'),
 Document(metadata={'source': '/Users/hyewonmac/study/modu_llm_5/m

In [8]:
# CSV 파일 로더
from langchain_community.document_loaders.csv_loader import CSVLoader

# 기본 파일 로드
csv_loader = CSVLoader('../data/kbo_teams_2023.csv')
csv_docs = csv_loader.load()

csv_docs
# print("처음 문서의 내용: \n", csv_docs[0].page_content)

[Document(metadata={'source': '../data/kbo_teams_2023.csv', 'row': 0}, page_content="Team: KIA 타이거즈\nCity: 광주\nFounded: 1982\nHome Stadium: 광주-기아 챔피언스 필드\nChampionships: 11\nIntroduction: KBO 리그의 전통 강호로, 역대 최다 우승 기록을 보유하고 있다. '타이거즈 스피릿'으로 유명하며, 양현종, 안치홍 등 스타 선수들을 배출했다. 광주를 연고로 하는 유일한 프로야구팀으로 지역 사랑이 강하다."),
 Document(metadata={'source': '../data/kbo_teams_2023.csv', 'row': 1}, page_content='Team: 두산 베어스\nCity: 서울\nFounded: 1982\nHome Stadium: 잠실야구장\nChampionships: 6\nIntroduction: 2015년부터 2019년까지 5년 연속 한국시리즈에 진출한 강팀이다. 김태형 감독 체제에서 체계적인 선수 육성으로 주목받았으며, 정수빈, 김재환 등 핵심 선수들의 활약이 돋보인다.'),
 Document(metadata={'source': '../data/kbo_teams_2023.csv', 'row': 2}, page_content="Team: SSG 랜더스\nCity: 인천\nFounded: 2000\nHome Stadium: 인천SSG랜더스필드\nChampionships: 5\nIntroduction: SK 와이번스에서 SSG 랜더스로 구단명을 변경했다. 2022년 와일드카드 결정전부터 한국시리즈까지 전승으로 우승을 차지하며 '노히트 행진'을 달성했다. 추신수, 김광현 등 스타 선수들이 포진해 있다."),
 Document(metadata={'source': '../data/kbo_teams_2023.csv', 'row': 3}, page_content='Team: 삼성 라이온즈\nCity: 대구\nFo

In [9]:
# CSV 파싱 커스터마이징
from langchain_community.document_loaders.csv_loader import CSVLoader

# 기본 파일 로드
csv_loader = CSVLoader(
	file_path='../data/kbo_teams_2023.csv',
	csv_args={
		'delimiter': ",", # 구분자 지정
		'quotechar': '"', # 따옴표 문자 지정
	},
	encoding="utf-8"
)
csv_docs = csv_loader.load()

csv_docs
# print("처음 문서의 내용: \n", csv_docs[0].page_content)


[Document(metadata={'source': '../data/kbo_teams_2023.csv', 'row': 0}, page_content="Team: KIA 타이거즈\nCity: 광주\nFounded: 1982\nHome Stadium: 광주-기아 챔피언스 필드\nChampionships: 11\nIntroduction: KBO 리그의 전통 강호로, 역대 최다 우승 기록을 보유하고 있다. '타이거즈 스피릿'으로 유명하며, 양현종, 안치홍 등 스타 선수들을 배출했다. 광주를 연고로 하는 유일한 프로야구팀으로 지역 사랑이 강하다."),
 Document(metadata={'source': '../data/kbo_teams_2023.csv', 'row': 1}, page_content='Team: 두산 베어스\nCity: 서울\nFounded: 1982\nHome Stadium: 잠실야구장\nChampionships: 6\nIntroduction: 2015년부터 2019년까지 5년 연속 한국시리즈에 진출한 강팀이다. 김태형 감독 체제에서 체계적인 선수 육성으로 주목받았으며, 정수빈, 김재환 등 핵심 선수들의 활약이 돋보인다.'),
 Document(metadata={'source': '../data/kbo_teams_2023.csv', 'row': 2}, page_content="Team: SSG 랜더스\nCity: 인천\nFounded: 2000\nHome Stadium: 인천SSG랜더스필드\nChampionships: 5\nIntroduction: SK 와이번스에서 SSG 랜더스로 구단명을 변경했다. 2022년 와일드카드 결정전부터 한국시리즈까지 전승으로 우승을 차지하며 '노히트 행진'을 달성했다. 추신수, 김광현 등 스타 선수들이 포진해 있다."),
 Document(metadata={'source': '../data/kbo_teams_2023.csv', 'row': 3}, page_content='Team: 삼성 라이온즈\nCity: 대구\nFo

In [1]:
# 소스 컬럼 지정
from langchain_community.document_loaders.csv_loader import CSVLoader

csv_loader = CSVLoader(
	file_path='../data/kbo_teams_2023.csv',
	metadata_columns=["City", "Founded"],
	content_columns=["Teams", "Introduction"],
	encoding="utf-8",
	source_column="Team" # 이 컬럼의 값이 메타데이터의 souce로 사용됨
)

csv_docs = csv_loader.load()

csv_docs

/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(metadata={'source': 'KIA 타이거즈', 'row': 0, 'City': '광주', 'Founded': '1982'}, page_content="Introduction: KBO 리그의 전통 강호로, 역대 최다 우승 기록을 보유하고 있다. '타이거즈 스피릿'으로 유명하며, 양현종, 안치홍 등 스타 선수들을 배출했다. 광주를 연고로 하는 유일한 프로야구팀으로 지역 사랑이 강하다."),
 Document(metadata={'source': '두산 베어스', 'row': 1, 'City': '서울', 'Founded': '1982'}, page_content='Introduction: 2015년부터 2019년까지 5년 연속 한국시리즈에 진출한 강팀이다. 김태형 감독 체제에서 체계적인 선수 육성으로 주목받았으며, 정수빈, 김재환 등 핵심 선수들의 활약이 돋보인다.'),
 Document(metadata={'source': 'SSG 랜더스', 'row': 2, 'City': '인천', 'Founded': '2000'}, page_content="Introduction: SK 와이번스에서 SSG 랜더스로 구단명을 변경했다. 2022년 와일드카드 결정전부터 한국시리즈까지 전승으로 우승을 차지하며 '노히트 행진'을 달성했다. 추신수, 김광현 등 스타 선수들이 포진해 있다."),
 Document(metadata={'source': '삼성 라이온즈', 'row': 3, 'City': '대구', 'Founded': '1982'}, page_content='Introduction: 2011년부터 2015년까지 5년 연속 한국시리즈 우승을 차지한 명문 구단이다. 이승엽, 박진만 등 레전드 선수를 배출했으며, 체계적인 프런트와 선수 육성 시스템으로 유명하다.'),
 Document(metadata={'source': 'LG 트윈스', 'row': 4, 'City': '서울', 'Founded': '1982'}, page_content='Introduc

In [2]:
# 소스 컬럼 지정
from langchain_community.document_loaders.csv_loader import CSVLoader

csv_loader = CSVLoader(
	file_path='../data/kbo_teams_2023.csv',
	metadata_columns=["City", "Founded"],
	content_columns=["Teams", "Introduction"],
	encoding="utf-8",
	# source_column="Team" # 이 컬럼의 값이 메타데이터의 souce로 사용됨
)

csv_docs = csv_loader.load()

csv_docs

[Document(metadata={'source': '../data/kbo_teams_2023.csv', 'row': 0, 'City': '광주', 'Founded': '1982'}, page_content="Introduction: KBO 리그의 전통 강호로, 역대 최다 우승 기록을 보유하고 있다. '타이거즈 스피릿'으로 유명하며, 양현종, 안치홍 등 스타 선수들을 배출했다. 광주를 연고로 하는 유일한 프로야구팀으로 지역 사랑이 강하다."),
 Document(metadata={'source': '../data/kbo_teams_2023.csv', 'row': 1, 'City': '서울', 'Founded': '1982'}, page_content='Introduction: 2015년부터 2019년까지 5년 연속 한국시리즈에 진출한 강팀이다. 김태형 감독 체제에서 체계적인 선수 육성으로 주목받았으며, 정수빈, 김재환 등 핵심 선수들의 활약이 돋보인다.'),
 Document(metadata={'source': '../data/kbo_teams_2023.csv', 'row': 2, 'City': '인천', 'Founded': '2000'}, page_content="Introduction: SK 와이번스에서 SSG 랜더스로 구단명을 변경했다. 2022년 와일드카드 결정전부터 한국시리즈까지 전승으로 우승을 차지하며 '노히트 행진'을 달성했다. 추신수, 김광현 등 스타 선수들이 포진해 있다."),
 Document(metadata={'source': '../data/kbo_teams_2023.csv', 'row': 3, 'City': '대구', 'Founded': '1982'}, page_content='Introduction: 2011년부터 2015년까지 5년 연속 한국시리즈 우승을 차지한 명문 구단이다. 이승엽, 박진만 등 레전드 선수를 배출했으며, 체계적인 프런트와 선수 육성 시스템으로 유명하다.'),
 Document(metadata={'source': 

5. Directory 로더

- 개요 : 디렉토리에서 파일들을 읽어 Document로 객체 변환
- 기능 
	1. 파일 시스템 탐색 (와일드 카드 패턴 지원)
	2. 멀티스레딩을 통한 파일 I/O
	3. 인코딩 오류 등의 예외처리
- glob 패턴 사용법
	- *.txt: 현재 디렉토리의 모든 txt 파일
	- **/*.txt: 모든 하위 디렉토리의 txt파일(재귀 검색)
	- data/*_2023.csv: data디렉토리의 _2023으로 끝나는 CSV파일

In [3]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader

# 기본 사용법
dir_loader = DirectoryLoader(
	path="../",	# 파일 경로 - 현재 디렉토리
	glob="**/*_2023.csv", # 파일 확장자 - txt파일만 로드
	loader_cls=CSVLoader,
	loader_kwargs={"encoding":"utf-8"}
)

dir_docs = dir_loader.load()

dir_docs

[Document(metadata={'source': '../data/kbo_teams_2023.csv', 'row': 0}, page_content="Team: KIA 타이거즈\nCity: 광주\nFounded: 1982\nHome Stadium: 광주-기아 챔피언스 필드\nChampionships: 11\nIntroduction: KBO 리그의 전통 강호로, 역대 최다 우승 기록을 보유하고 있다. '타이거즈 스피릿'으로 유명하며, 양현종, 안치홍 등 스타 선수들을 배출했다. 광주를 연고로 하는 유일한 프로야구팀으로 지역 사랑이 강하다."),
 Document(metadata={'source': '../data/kbo_teams_2023.csv', 'row': 1}, page_content='Team: 두산 베어스\nCity: 서울\nFounded: 1982\nHome Stadium: 잠실야구장\nChampionships: 6\nIntroduction: 2015년부터 2019년까지 5년 연속 한국시리즈에 진출한 강팀이다. 김태형 감독 체제에서 체계적인 선수 육성으로 주목받았으며, 정수빈, 김재환 등 핵심 선수들의 활약이 돋보인다.'),
 Document(metadata={'source': '../data/kbo_teams_2023.csv', 'row': 2}, page_content="Team: SSG 랜더스\nCity: 인천\nFounded: 2000\nHome Stadium: 인천SSG랜더스필드\nChampionships: 5\nIntroduction: SK 와이번스에서 SSG 랜더스로 구단명을 변경했다. 2022년 와일드카드 결정전부터 한국시리즈까지 전승으로 우승을 차지하며 '노히트 행진'을 달성했다. 추신수, 김광현 등 스타 선수들이 포진해 있다."),
 Document(metadata={'source': '../data/kbo_teams_2023.csv', 'row': 3}, page_content='Team: 삼성 라이온즈\nCity: 대구\nFo